# Extraction des données complètes pour le Rhones

In [1]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time

# URL de base de l'API
base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"

In [2]:
# Chargement du ficher des adresses du rhones.
df69 = pd.read_csv("data/adresses-69.csv", sep=";", low_memory=False)

In [3]:
# Extraction des codes postaux.
code_postaux = df69["code_postal"].unique()

# Methode de requête de l'API

In [4]:
# Liste des codes qui n'ont pas pu être récupérés.
missed_codes = list()

def query_api(params: dict, next_url: str | None = None, retries: int = 3, backoff: int = 2) -> list[dict]:
    """Récupère toutes les pages d'une requête API en boucle avec gestion des erreurs."""

    all_results = []

    recuperated = 0

    # Recuperation du code postal des paramètres.
    postal_code = params["qs"].split(":")[-1]

    while True:
        for attempt in range(retries):
            try:
                if next_url:
                    response = requests.get(url=next_url)
                else:
                    response = requests.get(url=base_url, params=params)

                if response.status_code == 200:

                    content = response.json()

                    # Monitoring
                    recuperated += (content["total"]- recuperated) if (content["total"] - recuperated) < params["size"] else params["size"]

                    

                    if content["total"] == 0:
                        print("No data found from the query.", flush=True)
                        return []
                
                    
                    all_results.extend(content["results"])
                    print(f"{postal_code}: {recuperated} éléments sur {content["total"]} ({int(recuperated*100/content["total"])}%)", flush=True)

                    next_url = content.get("next")
                    break  # sortie du retry loop si succès

                # Gestion d'erreur si requête invalide.
                else:
                    print(f"{postal_code}: Erreur de connexion avec l'API!, tentative {attempt+1}/{retries}", flush=True)
                    time.sleep(backoff * (attempt + 1))  # backoff progressif

            # Gestion d'erreur si problème de connexion.
            except requests.RequestException as e:
                print(f"{postal_code}: Erreur réseau: {e}, tentative {attempt+1}/{retries}", flush=True)
                time.sleep(backoff * (attempt + 1))  # backoff progressif

        # else de boucle for (si trop d'échecs.)
        else:
            print(f"{postal_code}: Échec permanent, abandon de cette requête.")

            # Addition du code manqué dans le dictionnaire.
            missed_codes.append(postal_code)

            return all_results # Retourner les informations récupérés jusqu'à présent.

        if not next_url:
            break

    return all_results

## Extraction normale

In [6]:
df_list = list()

for i, code in enumerate(code_postaux):

    print()
    print(f"==Récupération codes postaux: {i+1}/{len(code_postaux)}==")

    params = {
            "size": 1500,   
            "qs": f"code_postal_ban:{code}"
        }
    df_list.extend(query_api(params=params, retries=5))

# Concaténation de la liste de dictionnaire en dataframe.
df = pd.DataFrame(df_list)

df.to_csv("existants_69.csv")



==Récupération codes postaux: 1/91==
69790: 197 éléments sur 197 (100%)

==Récupération codes postaux: 2/91==
69170: 1500 éléments sur 2893 (51%)
69170: 2893 éléments sur 2893 (100%)

==Récupération codes postaux: 3/91==
69250: 1500 éléments sur 3167 (47%)
69250: 3000 éléments sur 3167 (94%)
69250: 3167 éléments sur 3167 (100%)

==Récupération codes postaux: 4/91==
69380: 1500 éléments sur 3216 (46%)
69380: 3000 éléments sur 3216 (93%)
69380: 3216 éléments sur 3216 (100%)

==Récupération codes postaux: 5/91==
69009: 1500 éléments sur 21052 (7%)
69009: 3000 éléments sur 21052 (14%)
69009: 4500 éléments sur 21052 (21%)
69009: 6000 éléments sur 21052 (28%)
69009: 7500 éléments sur 21052 (35%)
69009: 9000 éléments sur 21052 (42%)
69009: 10500 éléments sur 21052 (49%)
69009: 12000 éléments sur 21052 (57%)
69009: 13500 éléments sur 21052 (64%)
69009: 15000 éléments sur 21052 (71%)
69009: 16500 éléments sur 21052 (78%)
69009: 18000 éléments sur 21052 (85%)
69009: 19500 éléments sur 21052 (92

## Multitreading code

Le code finit en erreur, car trop de requêtes simultannées sont demandées à l'API (l'API coupe la connection).

In [ ]:
# def fetch_for_code(code):
#     """Exécute query_api pour un code postal donné."""
#     params = {
#         "size": 750,
#         "page": 1,
#         "qs": f"code_postal_ban:{code}"
#     }
#     return query_api(params=params, retries=5)


# # Liste des DataFrames pour tous les codes postaux
# with ThreadPoolExecutor(max_workers=10) as executor: 
#     all_lists = list(executor.map(fetch_for_code, code_postaux))

# list_to_df = []
# for sublist in all_lists:
#     list_to_df.extend(sublist)

# # Concaténation finale
# df_final = pd.DataFrame(list_to_df)

# # Ecriture.
# df_final.to_csv("existants_69.csv")


In [ ]:
# # Vérification de l'extraction.
# df = pd.read_csv("existants_69.csv", index_col=0)

# Batiment neuf

Utiliser l'URL suivante: https://data.ademe.fr/data-fair/api/v1/datasets/dpe02neuf/lines

In [ ]:
# Récupération des logements neufs.
new_habitation_list = list()

base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe02neuf/lines"

for i, code in enumerate(code_postaux):

    print()
    print(f"==Récupération codes postaux: {i+1}/{len(code_postaux)}==")

    params = {
            "size": 1500,   
            "qs": f"code_postal_ban:{code}"
        }

    new_habitation_list.extend(query_api(params=params))

# Concaténation de la liste de dictionnaire en dataframe.
df_news = pd.DataFrame(df_list)

# Ecriture du fichier en csv.
df_news.to_csv("neufs_69.csv")


69790: 1 éléments sur 1 (100%)
69170: 232 éléments sur 232 (100%)
69250: 480 éléments sur 480 (100%)
69380: 556 éléments sur 556 (100%)
69009: 1363 éléments sur 1363 (100%)
69008: 1500 éléments sur 3041 (49%)


KeyboardInterrupt: 